In [ ]:
import numpy as np, pandas as pd
import warnings
import psutil, os
warnings.filterwarnings('ignore')

# going on 1 decimal matching
path2 = 'Data/labelled_historical_modus_1_decimal_lat_long.csv'


# I'm going to sample 10% of values
# n = 10  # every 10th line = 10% of the lines
# df = pd.read_csv(path2, index_col=0, skiprows=lambda i: i % n != 0)


df = pd.read_csv(path2, index_col=0)

df.drop(columns = ['version', 'DATE'], inplace=True)

# initial memory usage
df.info(memory_usage='deep')

In [2]:
import psutil, os
psutil.Process(os.getpid()).memory_info()

pmem(rss=170303488, vms=880713728, shared=39342080, text=2342912, lib=0, data=200003584, dirty=0)

In [3]:
# memory downcasting system

def float_to_int(ser):
    try:
        int_ser = ser.astype(int)
        if (ser == int_ser).all():
            return int_ser
        else:
            return ser
    except ValueError:
        return ser
    
def all_float_to_int(df):
    df_to_use = df.copy()
    transform_fn = float_to_int
    condition = lambda x: list(x
                    .select_dtypes(include=["float"])
                    .columns)    
    
    return multi_assign(df_to_use, transform_fn, condition)

def multi_assign(df, transform_fn, condition):
    df_to_use = df.copy()
    
    return (df_to_use
        .assign(
            **{col: transform_fn(df_to_use[col])
               for col in condition(df_to_use)})
           )

def downcast_all(df, target_type, inital_type=None):
    #Gotta specify floats, unsigned, or integer
    #If integer, gotta be 'integer', not 'int'
    #Unsigned should look for Ints
    if inital_type is None:
        inital_type = target_type
    
    df_to_use = df.copy()
    
    transform_fn = lambda x: pd.to_numeric(x, 
                                downcast=target_type)
    
    condition = lambda x: list(x
                    .select_dtypes(include=[inital_type])
                    .columns) 
    
    return multi_assign(df_to_use, transform_fn, condition)

In [4]:
df = (df
     .pipe(all_float_to_int)
     .pipe(downcast_all, "float")
     .pipe(downcast_all, "integer")
     .pipe(downcast_all,  
           target_type = "unsigned", 
           inital_type = "integer")
)

In [5]:
# new memory usage
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 203729 entries, 9 to 2037289
Data columns (total 17 columns):
latitude      203729 non-null float32
longitude     203729 non-null float32
brightness    203729 non-null float32
scan          203729 non-null float32
track         203729 non-null float32
acq_time      203729 non-null uint16
satellite     203729 non-null object
confidence    203729 non-null uint8
bright_t31    203729 non-null float32
frp           203729 non-null float32
daynight      203729 non-null object
type          203729 non-null uint8
FIRE_YEAR     203729 non-null uint16
MONTH         203729 non-null uint8
isFire        203729 non-null uint8
WEEK          203729 non-null uint8
DAY           203729 non-null uint8
dtypes: float32(7), object(2), uint16(2), uint8(6)
memory usage: 33.7 MB


In [6]:
df['isFire'].value_counts()[1] / df['isFire'].value_counts()[0]

0.42074395380624285

In [7]:
random_state = 314

X = df.drop(columns = 'isFire')
y = df[['FIRE_YEAR', 'isFire']]


X_train = X[X['FIRE_YEAR'] <= 2014]
y_train = y[y['FIRE_YEAR'] <= 2014].drop(columns = 'FIRE_YEAR')

X_test = X[X['FIRE_YEAR'] > 2014]
y_test = y[y['FIRE_YEAR'] > 2014].drop(columns = 'FIRE_YEAR')

In [8]:
X_train.to_pickle('Data/X_train_small.pkl')
y_train.to_pickle('Data/y_train_small.pkl')
X_test.to_pickle('Data/X_test_small.pkl')
y_test.to_pickle('Data/y_test_small.pkl')